In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MiniProject6thSem/yolov7

/content/drive/.shortcut-targets-by-id/1b71hwMc2irBqih7QEhWaQsH2eMvIUCPY/MiniProject6thSem/yolov7


In [ ]:
!pip install -r requirements.txt
!pip install cython
!pip install scikit-image
!pip install opencv-python-headless
!pip install mediapipe
!pip install torch torchvision
!pip install ultralytics
!pip install easydict
!pip install numpy==1.23.4

In [ ]:
import cv2
import mediapipe as mp #face detector
import math
import numpy as np
import time
import warnings
warnings.simplefilter("ignore", UserWarning)
import torch
import torch.nn as  nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from pathlib import Path
from google.colab.patches import cv2_imshow
# Ensure that imports for DeepSort and YOLO are correctly handled
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, non_max_suppression, scale_coords, xyxy2xywh
from utils.torch_utils import select_device, time_synchronized
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort
# Ensure TensorFlow and Keras are correctly installed
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
torch.cuda.is_available()

False

In [ ]:
# Load pre-trained model
model = load_model('./pretrained_models/facialemotionmodel.h5')

# Load the cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + './pretrained_models/haarcascade_frontalface_default.xml')

# Define emotion labels
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Function to adjust predictions vector based on specified conditions
def adjust_predictions(predictions):
    # Copy the original predictions vector
    predictions = predictions.flatten()
    modified_predictions = predictions.copy()
    # Remove Disgust and add its probability to Neutral
    modified_predictions[4] += modified_predictions[1]
    modified_predictions[1] = 0

    # Remove Fear and add its probability to Neutral
    modified_predictions[4] += modified_predictions[2]
    modified_predictions[2] = 0

    # Remove Surprise and add its probability to Happy
    modified_predictions[3] += modified_predictions[6]
    modified_predictions[6] = 0

    # Remove Angry and add its probability to Sad
    modified_predictions[5] += modified_predictions[0]
    modified_predictions[0] = 0
    return modified_predictions

# Function to detect emotions from face image
def predict_emotion(face):
    # Resize image to match model input size
    face = cv2.resize(face, (48, 48))
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    face = img_to_array(face)  # Convert to array
    face = preprocess_input(face)  # Preprocess input for the model
    face = np.expand_dims(face, axis=0)  # Add batch dimension
    predictions = model.predict(face)
    # Adjust predictions
    modified_predictions = adjust_predictions(predictions)

    # Find the dominant emotion after adjustments
    emotion_index = np.argmax(modified_predictions)
    emotion = emotions[emotion_index]
    return emotion

In [ ]:
import math

def calculate_distance(keypoint1, keypoint2):
    """Calculate Euclidean distance between two keypoints."""
    return math.sqrt((keypoint2[0] - keypoint1[0])**2 + (keypoint2[1] - keypoint1[1])**2)

def calculate_angle(keypoint1, keypoint2, reference_vector):
    """Calculate the angle between two vectors with respect to a reference vector."""
    vector1 = (keypoint1[0] - keypoint2[0], keypoint1[1] - keypoint2[1])
    vector2 = (keypoint2[0] - reference_vector[0], keypoint2[1] - reference_vector[1])

    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude_product = math.sqrt((vector1[0]**2 + vector1[1]**2) * (vector2[0]**2 + vector2[1]**2))

    if magnitude_product == 0:
        return None

    cosine_angle = dot_product / magnitude_product
    angle_rad = math.acos(cosine_angle)
    angle_deg = math.degrees(angle_rad)

    return angle_deg

def get_midpoint(keypoint1, keypoint2):
    """Calculate the midpoint between two keypoints."""
    x_mid = (keypoint1[0] + keypoint2[0]) / 2
    y_mid = (keypoint1[1] + keypoint2[1]) / 2
    return x_mid, y_mid

def calculate_distances_and_angles(keypoints):
    """Calculate distances and angles between specified keypoints."""
    left_shoulder = keypoints[11]  # Keypoint index 11 is left shoulder

    left_elbow = keypoints[13] # Keypoint index 13 is left elbow


    left_wrist = keypoints[15] # Keypoint index 15 is left wrist
    right_shoulder = keypoints[12] # Keypoint index 12 is right shoulder
    right_elbow = keypoints[14]    # Keypoint index 14 is right elbow
    right_wrist = keypoints[16]    # Keypoint index 16 is right wrist
    nose = keypoints[0]            # Keypoint index 0 is nose
    neck = get_midpoint(left_shoulder, right_shoulder)  # Midpoint between left and right shoulders

    distances = {
        "left_shoulder_to_left_elbow": calculate_distance(left_shoulder, left_elbow),
        "left_elbow_to_left_wrist": calculate_distance(left_elbow, left_wrist),
        "right_shoulder_to_right_elbow": calculate_distance(right_shoulder, right_elbow),
        "right_elbow_to_right_wrist": calculate_distance(right_elbow, right_wrist),
        "nose_to_neck": calculate_distance(nose, neck)
    }

#     print(neck[0], neck[1] - 1)

    # angles = {
    #     "phi1": calculate_angle(nose, neck, (neck[0], neck[1] - 1)),  # Reference vector: straight up
    #     "phi2": calculate_angle(right_elbow, right_shoulder, (right_shoulder[0] + 1, right_shoulder[1])),  # Reference vector: pointing right
    #     "phi3": calculate_angle(right_wrist, right_elbow, (right_elbow[0], right_elbow[1] - 1)),  # Reference vector: straight up
    #     "phi4": calculate_angle(left_elbow, left_shoulder, (left_shoulder[0] - 1, left_shoulder[1])),  # Reference vector: pointing left
    #     "phi5": calculate_angle(left_wrist, left_elbow, (left_elbow[0], left_elbow[1] - 1))  # Reference vector: straight up
    # }

    angles = {
        "phi1": calculate_angle(nose, neck, (neck[0], neck[1] + 1)),  # Reference vector: perpendicular to the horizontal plane
        "phi2": calculate_angle(right_elbow, right_shoulder, (right_shoulder[0] + 1, right_shoulder[1])),  # Reference vector: pointing right
        "phi3": calculate_angle(right_wrist, right_elbow, (right_elbow[0], right_elbow[1] + 1)),  # Reference vector: horizontal
        "phi4": calculate_angle(left_elbow, left_shoulder, (left_shoulder[0] - 1, left_shoulder[1])),  # Reference vector: pointing left
        "phi5": calculate_angle(left_wrist, left_elbow, (left_elbow[0], left_elbow[1] + 1))  # Reference vector: horizontal
    }


    return distances, angles







In [ ]:
torch.cuda.device_count()

0

In [ ]:
def classify_learning_emotion(emotion, action):
    positive_emotions = ['Neutral', 'Happy', 'Surprise']
    negative_emotions = ['Sad', 'Fear', 'Disgust', 'Angry']
    attentive_postures = ['Looking_Forward', 'Raising_Hand', 'Reading', 'Writting']
    inattentive_postures = ['Sleeping', 'Turning_Around']
    if emotion in positive_emotions and action in attentive_postures:
        return "Focused"
    elif emotion in negative_emotions and action in attentive_postures:
        return "Confused"
    elif emotion in positive_emotions and action in inattentive_postures:
        return "Distracted"
    elif emotion in negative_emotions and action in inattentive_postures:
        return "Disengaged"
    else:
        return "Unknown"

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
labels = ['Looking_Forward', 'Raising_Hand', 'Reading', 'Sleeping', 'Turning_Around', 'Writting']
model_path = '/content/drive/MyDrive/Mini project 1 /pose action/csv of keypoints remove norm/model_remove_norm_1st.h5'  # Update with the path to your model
# model_path = '/content/drive/MyDrive/Mini project 1 /pose action/csv of modified dataset/model_40_fetarue_adam_0.001.h5'
action_model = load_model(model_path)


def setup_deepsort_and_yolo(weights_path, config_path, img_size=640):
    device = select_device(device='cpu' if not torch.cuda.is_available() else 'cuda')

    model = attempt_load(weights_path, map_location=device)
 #
    stride = int(model.stride.max())
    img_size = check_img_size(img_size, s=stride)
    cfg_deep = get_config()
    cfg_deep.merge_from_file(config_path)
    deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT, max_dist=cfg_deep.DEEPSORT.MAX_DIST,
                        min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE, nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE, max_age=cfg_deep.DEEPSORT.MAX_AGE,
                        n_init=cfg_deep.DEEPSORT.N_INIT, nn_budget=cfg_deep.DEEPSORT.NN_BUDGET, use_cuda=torch.cuda.is_available())
    print("Models loaded successfully")
    return model, deepsort, device, img_size

def process_video(input_video, output_video, weights_path, config_path):
    print("Initializing YOLO and DeepSort...")
    model, deepsort, device, img_size = setup_deepsort_and_yolo(weights_path, config_path)
    print("Processing Video...")
    vid_cap = cv2.VideoCapture(input_video)
    vid_writer = None
    frame_idx = 0
    # fourcc = int(vid_cap.get(cv2.CAP_PROP_FOURCC))
    # fps = vid_cap.get(cv2.CAP_PROP_FPS)
    while True:
        ret, frame = vid_cap.read()
        if not ret:
            print("No frames grabbed!")
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}...")
        # Resize frame while preserving the number of color channels and convert to RGB
        frame_resized = cv2.resize(frame, (img_size, img_size))
        # Convert frame to tensor with correct shape and number of channels
        frame_tensor = torch.from_numpy(frame_resized).to(device).permute(2, 0, 1).float().div(255.0).unsqueeze(0)
        with torch.no_grad():  # Add torch.no_grad() to prevent gradient computation
            pred = model(frame_tensor, augment=False)[0]
            pred = non_max_suppression(pred, 0.4, 0.5, classes=[0], agnostic=False)
        if len(pred):
            det = pred[0]
            if len(det):
                det[:, :4] = scale_coords(frame_tensor.shape[2:], det[:, :4], frame_resized.shape).round()
                xywhs = xyxy2xywh(det[:, :4])
                confs = det[:, 4]
                outputs = deepsort.update(xywhs.cpu(), confs.cpu(), frame_resized)
                if len(outputs):
                    for output in outputs:
                        bboxes = output[:4]
                        id = output[4]
                        x1, y1, x2, y2 = map(int, bboxes)
                        body_roi = frame_resized[y1:y2, x1:x2]
                        if body_roi.size > 0:
                            #detect_emotion(face_roi)
                            emotion = predict_emotion(body_roi)
                            pose = mp_pose.Pose(static_image_mode=False,
                                    model_complexity=2,
                                    smooth_landmarks=True,
                                    min_detection_confidence=0.5,
                                    min_tracking_confidence=0.5)

                            results = pose.process(cv2.cvtColor(body_roi, cv2.COLOR_BGR2RGB))
                            width = abs(x2-x1)
                            height = abs(y2-y1)
                            if results.pose_landmarks is None:
                                continue
                            # if results.pose_landmarks is not None:
                            #     mp_drawing.draw_landmarks(frame[y1:y2, x1:x2], results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                            # Extract keypoints and calculate features
                            keypoints = results.pose_landmarks.landmark
                            flattened_keypoints = [(kp.x, kp.y) for kp in keypoints[:25]]
                            distances, angles = calculate_distances_and_angles(flattened_keypoints)
                            sepearted_keypoints = [coord for kp in flattened_keypoints for coord in kp]
                            flattened_distances = list(distances.values())
                            flattened_angles = list(angles.values())
                            pose.close()



                            data_row = sepearted_keypoints + flattened_distances + flattened_angles

                            data_row = np.array(data_row).reshape(1, -1)
                            # Predict action
                            predicted_action = action_model.predict(data_row)
                            predicted_index = np.argmax(predicted_action[0])
                            predicted_label = labels[predicted_index]
                            # cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            # cv2.putText(frame_resized, f'ID {id}: {emotion} Action:{labels[predicted_index]}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                            learning_emotion = classify_learning_emotion(emotion,predicted_label)
                            overlay_text = f'ID {id}:{learning_emotion}'
                            print(f'Id :{id } :emotion:{emotion} action {predicted_label} le {learning_emotion}')

                            # Draw bounding box and overlay text
                            cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            text_size = cv2.getTextSize(overlay_text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
                            text_x = max(0, x1 - 5)
                            text_y = max(0, y1 - 5)
                            cv2.putText(frame_resized, overlay_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
# #idhar apna code add kar de
#                             cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                             cv2.putText(frame_resized, f'ID {id}: {emotion}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        if vid_writer is None and output_video:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            vid_writer = cv2.VideoWriter(output_video, fourcc, 30, (frame_resized.shape[1], frame_resized.shape[0]))
            print("Video writer initialized")
        if vid_writer is not None:
            vid_writer.write(frame_resized)
    vid_cap.release()
    if vid_writer:
        vid_writer.release()
        print("Video writer released")
    cv2.destroyAllWindows()
    print("Done processing video")
# Example usage
process_video('./input_data/tanmay_harsh_cut.mp4', './output_data/tanmay_harsh_cut_Ouput.mp4', './pretrained_models/yolov7_training.pt', './deep_sort_pytorch/configs/deep_sort.yaml')

# **Below this is the code for posture and the the integrated code**

In [ ]:
##integrated
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

def classify_learning_emotion(emotion, action):
    positive_emotions = ['Neutral', 'Happiness', 'Surprise']
    negative_emotions = ['Sadness', 'Fear', 'Disgust', 'Anger']
    attentive_postures = ['Looking_Forward', 'Raising_Hand', 'Reading', 'Writting']
    inattentive_postures = ['Sleeping', 'Turning_Around']
    if emotion in positive_emotions and action in attentive_postures:
        return "Focused"
    elif emotion in negative_emotions and action in attentive_postures:
        return "Confused"
    elif emotion in positive_emotions and action in inattentive_postures:
        return "Distracted"
    elif emotion in negative_emotions and action in inattentive_postures:
        return "Disengaged"
    else:
        return "Unknown"

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
labels = ['Looking_Forward', 'Raising_Hand', 'Reading', 'Sleeping', 'Turning_Around', 'Writting']
model_path = '/content/drive/MyDrive/Mini project 1 /pose action/csv of keypoints remove norm/model_remove_norm_1st.h5'  # Update with the path to your model
# model_path = '/content/drive/MyDrive/Mini project 1 /pose action/csv of modified dataset/model_40_fetarue_adam_0.001.h5'
action_model = load_model(model_path)

def setup_deepsort_and_yolo(weights_path, config_path, img_size=640):
    device = select_device(device='cpu' if not torch.cuda.is_available() else 'cuda')
    model = attempt_load(weights_path, map_location=device)
    stride = int(model.stride.max())
    img_size = check_img_size(img_size, s=stride)
    cfg_deep = get_config()
    cfg_deep.merge_from_file(config_path)
    deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT, max_dist=cfg_deep.DEEPSORT.MAX_DIST,
                        min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE, nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE, max_age=cfg_deep.DEEPSORT.MAX_AGE,
                        n_init=cfg_deep.DEEPSORT.N_INIT, nn_budget=cfg_deep.DEEPSORT.NN_BUDGET, use_cuda=torch.cuda.is_available())
    print("Models loaded successfully")
    return model, deepsort, device, img_size

def process_video(input_video, output_video, weights_path, config_path):
    print("Initializing YOLO and DeepSort...")
    model, deepsort, device, img_size = setup_deepsort_and_yolo(weights_path, config_path)
    print("Processing Video...")
    vid_cap = cv2.VideoCapture(input_video)
    vid_writer = None
    frame_idx = 0

    while True:
        ret, frame = vid_cap.read()
        if not ret:
            print("No frames grabbed!")
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}...")

        # Convert frame to tensor with correct shape and number of channels
        frame_tensor = torch.from_numpy(frame).to(device).permute(2, 0, 1).float().div(255.0).unsqueeze(0)

        with torch.no_grad():
            pred = model(frame_tensor, augment=False)[0]
            pred = non_max_suppression(pred, 0.4, 0.5, classes=[0], agnostic=False)

        if len(pred):
            det = pred[0]
            if len(det):
                det[:, :4] = scale_coords(frame_tensor.shape[2:], det[:, :4], frame.shape).round()
                xywhs = xyxy2xywh(det[:, :4])
                confs = det[:, 4]
                outputs = deepsort.update(xywhs.cpu(), confs.cpu(), frame)

                if len(outputs):
                    for output in outputs:
                        bboxes = output[:4]
                        id = output[4]
                        x1, y1, x2, y2 = map(int, bboxes)
                        body_roi = frame[y1:y2, x1:x2]

                        if body_roi.size > 0:
                            emotion = predict_emotion(body_roi)

                            pose = mp_pose.Pose(static_image_mode=False,
                                                model_complexity=2,
                                                smooth_landmarks=True,
                                                min_detection_confidence=0.5,
                                                min_tracking_confidence=0.5)

                            results = pose.process(cv2.cvtColor(body_roi, cv2.COLOR_BGR2RGB))

                            if results.pose_landmarks is not None:
                                keypoints = results.pose_landmarks.landmark
                                flattened_keypoints = [(kp.x, kp.y) for kp in keypoints[:25]]
                                distances, angles = calculate_distances_and_angles(flattened_keypoints)
                                sepearted_keypoints = [coord for kp in flattened_keypoints for coord in kp]
                                flattened_distances = list(distances.values())
                                flattened_angles = list(angles.values())
                                pose.close()

                                data_row = sepearted_keypoints + flattened_distances + flattened_angles
                                data_row = np.array(data_row).reshape(1, -1)

                                predicted_action = action_model.predict(data_row)
                                predicted_index = np.argmax(predicted_action[0])
                                predicted_label = labels[predicted_index]

                                learning_emotion = classify_learning_emotion(emotion, predicted_label)
                                overlay_text = f'ID {id}:{learning_emotion}'
                                print(f'Id :{id } :emotion:{emotion} action {predicted_label} le {learning_emotion}')

                                # Draw bounding box and overlay text on the original frame
                                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                text_size = cv2.getTextSize(overlay_text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
                                text_x = max(0, x1 - 5)
                                text_y = max(0, y1 - 5)
                                cv2.putText(frame, overlay_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        if vid_writer is None and output_video:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            vid_writer = cv2.VideoWriter(output_video, fourcc, 30, (frame.shape[1], frame.shape[0]))
            print("Video writer initialized")

        if vid_writer is not None:
            vid_writer.write(frame)

    vid_cap.release()

    if vid_writer:
        vid_writer.release()
        print("Video writer released")

    cv2.destroyAllWindows()
    print("Done processing video")

# Example usage
process_video('./input_data/test.mp4', './output_data/op1_combine2.mp4', './pretrained_models/yolov7_training.pt', './deep_sort_pytorch/configs/deep_sort.yaml')

# process_video('./input_data/3people.mp4', './output_data/output1.mp4', './pretrained_model/yolov7_training.pt', './deep_sort_pytorch/configs/deep_sort.yaml')